In [1]:
import os
from PIL import Image

In [2]:
src = '/media/sid/Seagate Expansion Drive/synchronization-images'
subdirs = [x[0] for x in os.walk(src)]
rgb_dirs = [x for x in subdirs if x.endswith('rgb')]
omni_dirs = [x for x in subdirs if x.endswith('omni')]
ir_dirs = [x for x in subdirs if x.endswith('ir')]
depth_dirs = [x for x in subdirs if x.endswith('depth')]

In [6]:
def crop_image(img, actual_width, actual_height):
    # Crop image to remove black borders
    left_width = (img.width - actual_width) // 2
    right_width = left_width + actual_width

    top_height = (img.height - actual_height) // 2
    bottom_height = top_height + actual_height

    img = img.crop((left_width, top_height, right_width, bottom_height))
    return img
    

def resize_image(img, imgtype='rgb'):
    # Resize RGB images to 1280x720
    # Resize Omni images to 1900x something
    if imgtype == 'rgb': 
        if img.size != (1280, 720):
            img = img.thumbnail((1280, 720),Image.ANTIALIAS) # Apparently preserves aspect ratio (need to test)
            return img
            # img = img.resize((1280, 720), Image.ANTIALIAS)

    elif imgtype == 'omni':
        img = crop_image(img, actual_width=1200, actual_height=900)
        return img
    
    elif imgtype == 'depth' or imgtype == 'ir':
        if img.size != (512, 424):
            img = crop_image(img, actual_width=512, actual_height=424)
            return img

    else:
        raise ValueError('Invalid image imgtype')

    return None

In [4]:
def resize_images_in_dir(dirs, imtype):
    for dir in dirs: 
        filetype = dir.split('/')[-1]
        dst = os.path.join(dir, 'resized')
        if not os.path.isdir(dst):
            os.mkdir(dst)
        files = [x for x in os.listdir(dir) if x.endswith('.jpg')]
        for file in files:
            img = Image.open(os.path.join(dir, file))
            img = resize_image(img, imtype)
            if img is not None:
                img.save(os.path.join(dst, file))

In [7]:
# resize_images_in_dir(rgb_dirs, 'rgb')
# print('resized rgb images')

resize_images_in_dir(omni_dirs, 'omni')
print('resized omni images')

resize_images_in_dir(ir_dirs, 'ir')
print('resized ir images')

resize_images_in_dir(depth_dirs, 'depth')
print('resized depth images')

In [ ]:
rgb_dirs